# AWS101 - Amazon S3 Select

## 1. Instalamos las dependencias

Instalamos la libreria boto3 que utilizaremos para interactuar con la API de AWS y Pandas que utilizaremos para manipular los datos, todo esto utilizando nuestra terminal.

In [ ]:
pip install boto3

## 2. inicializamos el cliente S3

importamos el la libreria e inicializamos el cliente de S3

In [ ]:
import boto3

s3 = boto3.client("s3")

## 3. ejecutamos la consulta

Utilizaremos el metodo `select_object_content` y vamos a capturar la respuesta en la variable `resp`.

In [ ]:
resp = s3.select_object_content(
    Bucket="aws101-s3-select",
    Key="agencias-viajes-20181012.csv",
    ExpressionType="SQL",
    Expression="SELECT * FROM s3object s",
    InputSerialization={"CSV": {"FileHeaderInfo": "Use"}, "CompressionType": "NONE"},
    OutputSerialization={"CSV": {}},
)

Apartir de aqui ya podemos hacer con los datos lo que mas nos guste.

En mi caso lo vamos a generar un **dataframe en pandas** para luego realizar algun analisis.

## 4. Consolidamos los datos

Consolidamos los datos de la respuesta en un Array y lo concatenamos una variable que luego utilizaremos para generar un `dataframe` con **Pandas**

In [ ]:
records = []
for event in resp["Payload"]:
    if "Records" in event:
        records.append(event["Records"]["Payload"])
    elif "Stats" in event:
        statsDetails = event["Stats"]["Details"]
        print("Stats details bytesScanned: ")
        print(statsDetails["BytesScanned"])
        print("Stats details bytesProcessed: ")
        print(statsDetails["BytesProcessed"])
        print("Stats details bytesReturned: ")
        print(statsDetails["BytesReturned"])

file_str = "".join(r.decode("utf-8") for r in records)

## 5. Creamos el Dataframe con Pandas

Importamos la libreria `pandas` como `PD`.

Tambien utilizaremos `StringIO` de la libreria `io`

Luego con `StringIO` utilizaremos el contenido de la variable `file_str`, donde concatenamos nuestros datos, para generar pasarle levantar los datos en `pandas` utilizando el metodo `read_csv` y lo guardaremos en la variable `select_df`.

Utilizando `PD.DataFrame` vamos a generar el dataframe que almacenaremos en la variables `df`

Finalmente vamos a listar las primeras lineas de nuestro dataframe utilizando `head`

In [ ]:
import pandas as PD
from io import StringIO

select_df = PD.read_csv(StringIO(file_str))
df = PD.DataFrame(select_df)
df.head(3)

### Agergamos el nombre de las columnas

Cuando definimos el query inicial con S3 elect le indicamos que nuestro archivo tenia encabezados de la primera linea y que los podia utilizar, pero esto causo que los resultados obtenidos no poseen el encabezado en la primera linea.

Por esta razon los vamos a importar dentro de nuestro dataframe

In [ ]:
resp = s3.select_object_content(
    Bucket="aws101-s3-select",
    Key="agencias-viajes-20181012.csv",
    ExpressionType="SQL",
    Expression="SELECT * FROM s3object s limit 1",
    InputSerialization={"CSV": {"FileHeaderInfo": "NONE"}, "CompressionType": "NONE"},
    OutputSerialization={"CSV": {}},
)
headers = ""
for event in resp["Payload"]:
    if "Records" in event:
        headers = (
            event["Records"]["Payload"].decode("utf-8", "ingore").strip().split(",")
        )
    elif "Stats" in event:
        statsDetails = event["Stats"]["Details"]
df.columns = headers
df.columns

In [ ]:
df.head(5)

In [ ]:
df["tipo"].value_counts()

In [ ]:
df = PD.get_dummies(df, columns=["tipo"])

In [ ]:
df.head(5)